In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import json
import os
import glob
import numpy as np

sns.set_context("talk", font_scale=1.0)

# Directory containing the JSON files
directory_path = 'validate/'
directory_weather_path = 'validate_weather/'
# List to store the processed data
all_experiments = []
weather_experiments = []

# Iterate through all JSON files in the directory
for file_path in glob.glob(os.path.join(directory_path, '*/[jlx]_metrics_*.json'), recursive=True):
    with open(file_path, 'r') as file:
        data = json.load(file)
    
    # Extract the model name from the file path
    model_name = os.path.basename(os.path.dirname(file_path)).split("_")[0]
    if model_name == "maskflownet" and os.path.basename(os.path.dirname(file_path)).split("_")[1] == "s":
        model_name = "maskflownet_s"
    dataset_name = os.path.basename(file_path).split("_")[2].split(".")[0]
    
    for experiment in data['experiments']:
        # Flatten the dictionary to include metrics at the top level
        experiment_flat = {**experiment, **experiment.pop('metrics')}
        # Add the model name and dataset name to the experiment data
        experiment_flat['model'] = model_name
        experiment_flat['dataset'] = dataset_name
        # Append the processed experiment to the list
        all_experiments.append(experiment_flat)



for file_path in glob.glob(os.path.join(directory_weather_path, '**/metrics_*.json'), recursive=True):
    with open(file_path, 'r') as file:
        data = json.load(file)
    
    # Extract the model name from the file path
    model_name = os.path.basename(os.path.dirname(file_path)).split("_")[0]
    if model_name == "maskflownet" and os.path.basename(os.path.dirname(file_path)).split("_")[1] == "s":
        model_name = "maskflownet_s"
    dataset_name = os.path.basename(file_path).split("_")[2].split(".")[0]
    weather_type = os.path.basename(os.path.dirname(os.path.dirname(file_path)))
    
    for experiment in data['experiments']:
        # Flatten the dictionary to include metrics at the top level
        experiment_flat = {**experiment, **experiment.pop('metrics')}
        # Add the model name and dataset name to the experiment data
        experiment_flat['model'] = model_name
        experiment_flat['dataset'] = dataset_name
        experiment_flat['weather_type'] = weather_type
        # Append the processed experiment to the list
        #all_experiments.append(experiment_flat)
        weather_experiments.append(experiment_flat)


df_weather = pd.DataFrame(weather_experiments)
df_weather.drop("metrics", axis="columns", inplace=True)
df_weather.drop("data", axis="columns", inplace=True)
df_weather['end_time'] = pd.to_datetime(df_weather['end_time'])

# Sort the DataFrame by 'start_time'
df_weather = df_weather.sort_values(by='end_time')
#df.to_csv("big_df.csv")
# Drop duplicates, keeping the most recent entry for each combination of specified columns
unique_columns = ['model', 'checkpoint', 'attack','loss', 'optimizer', 'learn', 'alph', 'rendering', 'transparency', 'depth', 'scene', 'recolor', 'do', 'motionblur', 'flakesize', 'constant' , 'motion', 'flake', 'frame', 'no', 'lr', 'unregistered', 'num', 'targeted', 'target', 'dataset']
df_weather = df_weather.drop_duplicates(subset=unique_columns, keep='last')
#df_weather.to_csv("test.csv")

# Replace optim=NaN with ground truth

# Replace ms with ms_raft+
df_weather['model'] = df_weather['model'].replace('ms', 'ms_raft+')

df_weather['model'] = df_weather['model'].replace('videoflow', 'videoflow_bof')

df_weather['model'] = df_weather['model'].replace('irr', 'irr_pwc')

df_weather.loc[(df_weather["dataset"] == "kitti-2015") & (df_weather["model"] == "liteflownet3"), "model"] = "liteflownet3_pseudoreg"




# Create a DataFrame from the combined data
df = pd.DataFrame(all_experiments)
df.drop("metrics", axis="columns", inplace=True)




# Set iterations and epsilon to 0 where 'attack' is 'none'
df.loc[df['attack'] == 'none', 'iterations'] = 0
df.loc[df['attack'] == 'none', 'epsilon'] = 0
# Replace 'epe_orig_preds' with 'epe' where 'attack' is 'none'
df.loc[df['attack'] == 'none', 'epe_orig_preds'] = df['epe']
# Replace 'epe_ground_truth' with 'epe' where 'attack' is 'none'
df.loc[df['attack'] == 'none', 'epe_ground_truth'] = df['epe']

# Get unique attack types excluding 'none'
attack_types = df['attack'].unique()
attack_types = attack_types[attack_types != 'none']

# Filter entries with attack == 'none'
none_entries = df[df['attack'] == 'none']

# Create copies of 'none' entries for each attack type
new_entries = []
for attack in attack_types:
    temp = none_entries.copy()
    temp['attack'] = attack
    new_entries.append(temp)

# Combine all new entries into a single DataFrame
new_entries_df = pd.concat(new_entries)

# Combine the new entries with the original DataFrame
result_df = pd.concat([df, new_entries_df])

df = df.rename(columns={'corruption': '3dcc_corruption', 'intensity': '3dcc_intensity'})
# Keep only most recent results
# Convert 'start_time' to datetime
df['end_time'] = pd.to_datetime(df['end_time'])

# Sort the DataFrame by 'start_time'
df = df.sort_values(by='end_time')
#df.to_csv("big_df.csv")
# Drop duplicates, keeping the most recent entry for each combination of specified columns
unique_columns = ['model', 'checkpoint', 'attack', 'targeted', 'target', 'dataset', 'norm', 'epsilon', 'iterations', 'alpha', 'optim', 'name', 'severity', '3dcc_corruption', '3dcc_intensity']
df = df.drop_duplicates(subset=unique_columns, keep='last')


# Replace optim=NaN with ground truth
df['optim'] = df['optim'].fillna("ground_truth")

# Replace ms with ms_raft+
df['model'] = df['model'].replace('ms', 'ms_raft+')

df['model'] = df['model'].replace('videoflow', 'videoflow_bof')

df['model'] = df['model'].replace('irr', 'irr_pwc')

df.loc[(df["dataset"] == "kitti-2015") & (df["model"] == "liteflownet3"), "model"] = "liteflownet3_pseudoreg"


# Find the missing columns in each DataFrame
df_missing_cols = [col for col in df_weather.columns if col not in df.columns]
df_weather_missing_cols = [col for col in df.columns if col not in df_weather.columns]

# Add missing columns to df and fill them with NaN
for col in df_missing_cols:
    df[col] = np.nan

# Add missing columns to df_weather and fill them with NaN
for col in df_weather_missing_cols:
    df_weather[col] = np.nan

# Reorder columns in both dataframes to match (optional, but helpful for clean appending)
df_weather = df_weather[df.columns]  # reorder df columns to match df_weather

# Append the two DataFrames
df = pd.concat([df, df_weather], ignore_index=True)

#df.to_csv("big_df.csv")
big_df = df

# Directory containing the JSON files
directory_path = 'validate/'

# List to store the processed data
all_experiments = []

# Iterate through all JSON files in the directory
for file_path in glob.glob(os.path.join(directory_path, '*/[jlx]_iteration_metrics_*.json'), recursive=True):
    with open(file_path, 'r') as file:
        data = json.load(file)
    
    # Extract the model name from the file path
    model_name = os.path.basename(os.path.dirname(file_path)).split("_")[0]
    if model_name == "maskflownet" and os.path.basename(os.path.dirname(file_path)).split("_")[1] == "s":
        model_name = "maskflownet_s"
    dataset_name = os.path.basename(file_path).split("_")[3].split(".")[0]
    
    for experiment in data['experiments']:
        # Flatten the dictionary to include metrics at the top level
        experiment_flat = {**experiment, **experiment.pop('metrics')}
        # Add the model name and dataset name to the experiment data
        experiment_flat['model'] = model_name
        experiment_flat['dataset'] = dataset_name
        # Append the processed experiment to the list
        experiment_flat_key_rename = {}
        for key, value in experiment_flat.items():
            if key.startswith("epe_ground_truth_"):
                new_key = key.replace("epe_ground_truth_", "epe_gt_")
                experiment_flat_key_rename[new_key] = value
            else:
                experiment_flat_key_rename[key] = value
        #print(experiment_flat_key_rename)
        all_experiments.append(experiment_flat_key_rename)

# Create a DataFrame from the combined data
iterations_df = pd.DataFrame(all_experiments)
iterations_df.drop("metrics", axis="columns", inplace=True)

# Keep only most recent results
# Convert 'start_time' to datetime
iterations_df['end_time'] = pd.to_datetime(df['end_time'])

# Sort the DataFrame by 'start_time'
iterations_df = iterations_df.sort_values(by='end_time')

# Drop duplicates, keeping the most recent entry for each combination of specified columns
unique_columns = ['model', 'checkpoint', 'attack', 'targeted', 'target', 'dataset', 'norm', 'epsilon', 'iterations', 'alpha', 'optim']
iterations_df = iterations_df.drop_duplicates(subset=unique_columns, keep='last')



# Add i0 to the iterations dataframe
none_df = df[df['attack'] == 'none']

# Select only the necessary columns for the join
none_df = none_df[['model', 'checkpoint', 'dataset', 'epe', "epe_initial_to_negative", "epe_initial_to_zero"]]
none_df.rename(columns={'epe': 'epe_gt_i0'}, inplace=True)

iterations_df.to_csv("iteration_df.csv")

# Perform the join operation to add `epe_i0` to `iterations_df`
iterations_df = pd.merge(iterations_df, none_df, on=['model', 'checkpoint', 'dataset'], how='left')

iterations_df['epe_target_i0'] = iterations_df.apply(
    lambda row: row['epe_initial_to_negative'] if row['target'] == 'negative' else row['epe_initial_to_zero'],
    axis=1
)


iterations_df.to_csv("iteration_df.csv")

# Sort the dataframe by 'start_time' to ensure it remains ordered
iterations_df = iterations_df.sort_values(by='start_time')

iterations_df = iterations_df.dropna(subset=['epe_gt_i20'])
# Replace optim=NaN with ground truth
iterations_df['optim'] = iterations_df['optim'].fillna("ground_truth")

# Replace ms with ms_raft+
iterations_df['model'] = iterations_df['model'].replace('ms', 'ms_raft+')

# Replace ms with ms_raft+
iterations_df['model'] = iterations_df['model'].replace('videoflow', 'videoflow_bof')

iterations_df['model'] = iterations_df['model'].replace('irr', 'irr_pwc')

iterations_df.loc[(iterations_df["dataset"] == "kitti-2015") & (iterations_df["model"] == "liteflownet3"), "model"] = "liteflownet3_pseudoreg"

# Display the updated dataframe
#iterations_df.to_csv("iteration_df.csv")

# Add missing columns to df1 with NaN values
#for col in missing_columns:
#    big_df[col] = np.nan
# Key columns excluding 'start_time' and 'end_time'
key_columns = ['model', 'checkpoint', 'attack', 'norm', 'epsilon',
               'targeted', 'target', 'loss', 'dataset',
               'iterations', 'alpha',
               'optim', 'boxconstraint']

# Perform an outer merge on all key columns
merged_df = pd.merge(iterations_df, big_df, how='outer', on=key_columns, suffixes=('_iter', '_big'))

# Retain only 'start_time' and 'end_time' from 'big_df'
# First, rename the 'start_time_big' and 'end_time_big' to 'start_time' and 'end_time'
merged_df['start_time'] = merged_df['start_time_big']
merged_df['end_time'] = merged_df['end_time_big']
merged_df['duration'] = merged_df['duration_big']
#merged_df['epe_initial_to_negative'] = merged_df['epe_initital_to_zero']

# Drop the other 'start_time' and 'end_time' columns from 'iterations_df' (i.e., '_iter' suffixed columns)
merged_df.drop(columns=['start_time_iter', 'end_time_iter', 'start_time_big', 'end_time_big', 'duration_big', 'duration_iter'], inplace=True)

# The resulting DataFrame will now have only the 'start_time' and 'end_time' from big_df
columns_to_process = ['epe_initial_to_zero', 'epe_initial_to_negative']

# Loop through each column and apply the logic
for col in columns_to_process:
    # Create the new column without suffix
    merged_df[col] = merged_df[col + '_big'].combine_first(merged_df[col + '_iter'])
    
    # Drop the _big and _iter columns now that we've merged them
    merged_df.drop(columns=[col + '_big', col + '_iter'], inplace=True)

# Now the dataframe contains only the new merged columns without suffixes
print(merged_df)
def map_dataset_values(value):
    if value == 'final':
        return 'sintel-final'
    elif value == 'clean':
        return 'sintel-clean'
    elif value == '2015':
        return 'kitti-2015'
    else:
        return value  # Leave the value unchanged if it doesn't match

# Apply the function to the 'dataset' column
merged_df['dataset'] = merged_df['dataset'].apply(map_dataset_values)

merged_df = merged_df.sort_values(by=['model', 'dataset', 'norm', 'attack'], ascending=True)



merged_df.to_csv("one_single.csv")

weather_df = merged_df[merged_df['attack'] == 'weather']

# Save the filtered data to a CSV file
weather_df.to_csv('weather_data.csv', index=False)

              model checkpoint   attack norm  epsilon  iterations  alpha  \
0              ccmr      kitti     3dcc  NaN      NaN         NaN    NaN   
1              ccmr      kitti     3dcc  NaN      NaN         NaN    NaN   
2              ccmr      kitti     3dcc  NaN      NaN         NaN    NaN   
3              ccmr      kitti     3dcc  NaN      NaN         NaN    NaN   
4              ccmr      kitti     3dcc  NaN      NaN         NaN    NaN   
...             ...        ...      ...  ...      ...         ...    ...   
7344  videoflow_bof     sintel  weather  NaN      NaN         NaN    NaN   
7345  videoflow_bof     sintel  weather  NaN      NaN         NaN    NaN   
7346  videoflow_bof     sintel  weather  NaN      NaN         NaN    NaN   
7347  videoflow_bof     sintel  weather  NaN      NaN         NaN    NaN   
7348  videoflow_bof     sintel  weather  NaN      NaN         NaN    NaN   

      targeted    target loss  ...   no   lr  unregistered     num  \
0        False   

In [20]:
import numpy as np

combinations_list = [
    #Targeted: True, epsilon: 0.0314, target: 'negative' or 'zero' (for attacks: pgd, cospgd, bim, norm: 'inf')
    # {'attack': 'pgd', 'targeted': True, 'iterations': 20.0, 'alpha': 0.01, 'loss': 'epe', 'epsilon': 0.0314, 'target': 'negative', 'norm': 'inf'},
    # {'attack': 'pgd', 'targeted': True, 'iterations': 20.0, 'alpha': 0.01, 'loss': 'epe', 'epsilon': 0.0314, 'target': 'zero', 'norm': 'inf'},
    # {'attack': 'cospgd', 'targeted': True, 'iterations': 20.0, 'alpha': 0.01, 'loss': 'epe', 'epsilon': 0.0314, 'target': 'negative', 'norm': 'inf'},
    # {'attack': 'cospgd', 'targeted': True, 'iterations': 20.0, 'alpha': 0.01, 'loss': 'epe', 'epsilon': 0.0314, 'target': 'zero', 'norm': 'inf'},
    # {'attack': 'bim', 'targeted': True, 'iterations': 20.0, 'alpha': 0.01, 'loss': 'epe', 'epsilon': 0.0314, 'target': 'negative', 'norm': 'inf'},
    # {'attack': 'bim', 'targeted': True, 'iterations': 20.0, 'alpha': 0.01, 'loss': 'epe', 'epsilon': 0.0314, 'target': 'zero', 'norm': 'inf'},

    # # Targeted: True, epsilon: 0.2510, target: 'negative' or 'zero' (for attacks: pgd, cospgd, bim, norm: 'two')
    # {'attack': 'pgd', 'targeted': True, 'iterations': 20.0, 'alpha': 0.1, 'loss': 'epe', 'epsilon': 0.2510, 'target': 'negative', 'norm': 'two'},
    # {'attack': 'pgd', 'targeted': True, 'iterations': 20.0, 'alpha': 0.1, 'loss': 'epe', 'epsilon': 0.2510, 'target': 'zero', 'norm': 'two'},
    # {'attack': 'cospgd', 'targeted': True, 'iterations': 20.0, 'alpha': 0.1, 'loss': 'epe', 'epsilon': 0.2510, 'target': 'negative', 'norm': 'two'},
    # {'attack': 'cospgd', 'targeted': True, 'iterations': 20.0, 'alpha': 0.1, 'loss': 'epe', 'epsilon': 0.2510, 'target': 'zero', 'norm': 'two'},
    # {'attack': 'bim', 'targeted': True, 'iterations': 20.0, 'alpha': 0.1, 'loss': 'epe', 'epsilon': 0.2510, 'target': 'negative', 'norm': 'two'},
    # {'attack': 'bim', 'targeted': True, 'iterations': 20.0, 'alpha': 0.1, 'loss': 'epe', 'epsilon': 0.2510, 'target': 'zero', 'norm': 'two'},

    # # Targeted: True, epsilon: 0.0314, target: 'negative' or 'zero' (for attack: fgsm, norm: 'inf')
    # {'attack': 'fgsm', 'targeted': True, 'loss': 'epe', 'epsilon': 0.0314, 'target': 'negative', 'norm': 'inf'},
    # {'attack': 'fgsm', 'targeted': True, 'loss': 'epe', 'epsilon': 0.0314, 'target': 'zero', 'norm': 'inf'},

    # # Targeted: True, epsilon: 0.2510, target: 'negative' or 'zero' (for attack: fgsm, norm: 'two')
    # {'attack': 'fgsm', 'targeted': True, 'loss': 'epe', 'epsilon': 0.2510, 'target': 'negative', 'norm': 'two'},
    # {'attack': 'fgsm', 'targeted': True, 'loss': 'epe', 'epsilon': 0.2510, 'target': 'zero', 'norm': 'two'},

    # # Targeted: False, epsilon: 0.0314 or 0.0157, target: None (for attacks: pgd, cospgd, bim, norm: 'inf')
    # {'attack': 'pgd', 'targeted': False, 'iterations': 20.0, 'alpha': 0.01, 'loss': 'epe', 'epsilon': 0.0314, 'norm': 'inf'},
    # {'attack': 'pgd', 'targeted': False, 'iterations': 20.0, 'alpha': 0.01, 'loss': 'epe', 'epsilon': 0.0157, 'norm': 'inf'},
    # {'attack': 'cospgd', 'targeted': False, 'iterations': 20.0, 'alpha': 0.01, 'loss': 'epe', 'epsilon': 0.0314, 'norm': 'inf'},
    # {'attack': 'cospgd', 'targeted': False, 'iterations': 20.0, 'alpha': 0.01, 'loss': 'epe', 'epsilon': 0.0157, 'norm': 'inf'},
    # {'attack': 'bim', 'targeted': False, 'iterations': 20.0, 'alpha': 0.01, 'loss': 'epe', 'epsilon': 0.0314, 'norm': 'inf'},
    # {'attack': 'bim', 'targeted': False, 'iterations': 20.0, 'alpha': 0.01, 'loss': 'epe', 'epsilon': 0.0157, 'norm': 'inf'},

    # # Targeted: False, epsilon: 0.2510, target: None (for attacks: pgd, cospgd, bim, norm: 'two')
    # {'attack': 'pgd', 'targeted': False, 'iterations': 20.0, 'alpha': 0.1, 'loss': 'epe', 'epsilon': 0.2510, 'norm': 'two'},
    # {'attack': 'cospgd', 'targeted': False, 'iterations': 20.0, 'alpha': 0.1, 'loss': 'epe', 'epsilon': 0.2510, 'norm': 'two'},
    # {'attack': 'bim', 'targeted': False, 'iterations': 20.0, 'alpha': 0.1, 'loss': 'epe', 'epsilon': 0.2510, 'norm': 'two'},

    # #Top 9 models configuration
    # {'targeted': False, 'attack': 'cospgd', 'epsilon': 0.0314, 'alpha': 0.01, 'iterations': 20.0, 'norm': 'inf', 'loss': 'epe', 'optim': "initial_flow"},
    # {'targeted': False, 'attack': 'pgd', 'epsilon': 0.0314, 'alpha': 0.01, 'iterations': 20.0, 'norm': 'inf', 'loss': 'epe', 'optim': "initial_flow"},

    # # Targeted: False, epsilon: 0.0314 or 0.0157, target: None (for attack: fgsm, norm: 'inf')
    # {'attack': 'fgsm', 'targeted': False, 'loss': 'epe', 'epsilon': 0.0314, 'norm': 'inf'},
    # {'attack': 'fgsm', 'targeted': False, 'loss': 'epe', 'epsilon': 0.0157, 'norm': 'inf'},

    # # Targeted: False, epsilon: 0.2510, target: None (for attack: fgsm, norm: 'two')
    # {'attack': 'fgsm', 'targeted': False, 'loss': 'epe', 'epsilon': 0.2510, 'norm': 'two'},

    {'attack': 'common_corruptions', 'name': 'gaussian_noise', 'severity': 3.0},
    {'attack': 'common_corruptions', 'name': 'shot_noise', 'severity': 3.0},
    {'attack': 'common_corruptions', 'name': 'impulse_noise', 'severity': 3.0},
    {'attack': 'common_corruptions', 'name': 'defocus_blur', 'severity': 3.0},
    {'attack': 'common_corruptions', 'name': 'glass_blur', 'severity': 3.0},
    {'attack': 'common_corruptions', 'name': 'motion_blur', 'severity': 3.0},
    {'attack': 'common_corruptions', 'name': 'zoom_blur', 'severity': 3.0},
    {'attack': 'common_corruptions', 'name': 'snow', 'severity': 3.0},
    {'attack': 'common_corruptions', 'name': 'frost', 'severity': 3.0},
    {'attack': 'common_corruptions', 'name': 'fog', 'severity': 3.0},
    {'attack': 'common_corruptions', 'name': 'brightness', 'severity': 3.0},
    {'attack': 'common_corruptions', 'name': 'contrast', 'severity': 3.0},
    {'attack': 'common_corruptions', 'name': 'elastic_transform', 'severity': 3.0},
    {'attack': 'common_corruptions', 'name': 'pixelate', 'severity': 3.0},
    {'attack': 'common_corruptions', 'name': 'jpeg_compression', 'severity': 3.0},

    {'attack': '3dcc', '3dcc_corruption': 'far_focus', '3dcc_intensity': 3.0},
    {'attack': '3dcc', '3dcc_corruption': 'near_focus', '3dcc_intensity': 3.0},
    {'attack': '3dcc', '3dcc_corruption': 'fog_3d', '3dcc_intensity': 3.0},
    {'attack': '3dcc', '3dcc_corruption': 'color_quant', '3dcc_intensity': 3.0},
    {'attack': '3dcc', '3dcc_corruption': 'iso_noise', '3dcc_intensity': 3.0},
    {'attack': '3dcc', '3dcc_corruption': 'low_light', '3dcc_intensity': 3.0},
    {'attack': '3dcc', '3dcc_corruption': 'xy_motion_blur', '3dcc_intensity': 3.0},
    {'attack': '3dcc', '3dcc_corruption': 'z_motion_blur', '3dcc_intensity': 3.0},

    # {'attack': 'weather', 'targeted': False, 'flakesize': 71.0, 'depth': 9.0, 'transparency': 0.75, 'do': 0.0, 'weather_type': 'snow'},
    # {'attack': 'weather', 'targeted': True, 'target': 'negative', 'flakesize': 71.0, 'depth': 9.0, 'transparency': 0.75, 'do': 0.0, 'weather_type': 'snow'},
    # {'attack': 'weather', 'targeted': True, 'target': 'zero', 'flakesize': 71.0, 'depth': 9.0, 'transparency': 0.75, 'do': 0.0, 'weather_type': 'snow'},

    # {'attack': 'weather', 'targeted': False, 'num': 1000.0, 'flakesize': 51.0, 'depth': 9.0, 'transparency': 0.75, 'do': 1.0,  'weather_type': 'rain'},
    # {'attack': 'weather', 'targeted': True, 'target': 'negative',  'num': 1000.0, 'flakesize': 51.0, 'depth': 9.0, 'transparency': 0.75, 'do': 1.0, 'weather_type': 'rain'},
    # {'attack': 'weather', 'targeted': True,'target': 'zero',  'num': 1000.0, 'flakesize': 51.0, 'depth': 9.0, 'transparency': 0.75, 'do': 1.0, 'weather_type': 'rain'},

    # {'attack': 'weather', 'targeted': False, 'num': 60.0, 'flakesize': 451.0, 'depth': 0.8, 'constant': 0.3, 'transparency': 0.25, 'do': 0.0,  'weather_type': 'fog'},
    # {'attack': 'weather', 'targeted': True, 'target': 'negative',  'num': 60.0, 'flakesize': 451.0, 'depth': 0.8, 'constant': 0.3, 'transparency': 0.25, 'do': 0.0, 'weather_type': 'fog'},
    # {'attack': 'weather', 'targeted': True, 'target': 'zero',  'num': 60.0, 'flakesize': 451.0, 'depth': 0.8, 'constant': 0.3, 'transparency': 0.25, 'do': 0.0, 'weather_type': 'fog'},
    
    # {'attack': 'weather', 'targeted': False, 'num': 3000.0, 'flakesize': 41.0, 'depth': 9.0, 'transparency': 1.5, 'do': 1.0,  'weather_type': 'sparks'},
    # {'attack': 'weather', 'targeted': True, 'target': 'negative',  'num': 3000.0, 'flakesize': 41.0, 'depth': 9.0, 'transparency': 1.5, 'do': 1.0, 'weather_type': 'sparks'},
    # {'attack': 'weather', 'targeted': True, 'target': 'zero',  'num': 3000.0, 'flakesize': 41.0, 'depth': 9.0, 'transparency': 1.5, 'do': 1.0, 'weather_type': 'sparks'},

    # {'attack': 'none'},
    # {'attack': 'pcfa', 'targeted': True, 'target': 'negative', 'epsilon': 0.05, 'iterations': 20.0, 'alpha': 0.0},
    # {'attack': 'pcfa', 'targeted': True, 'target': 'zero', 'epsilon': 0.05, 'iterations': 20.0, 'alpha': 0.0},

]

# Set the dataset and norm
#datasets = ["sintel-final", "sintel-clean", "kitti-2015"]
# datasets = ["kitti-2015"]
datasets = ["sintel-final", "sintel-clean"]

# Define the datasets and corresponding model names
kitti_model_names = [
    "raft", "gma", "rpknet", "ccmr", "craft", "csflow", "dicl", "dip", 
    "fastflownet", "maskflownet", "flow1d", "flowformer", "flowformer++", 
    "gmflow", "gmflownet", "hd3", "irr_pwc", "liteflownet", 
    "liteflownet3_pseudoreg", "llaflow", "ms_raft+", 
    "rapidflow", "scopeflow", "skflow", 
    "starflow", "videoflow_bof"
]

sintel_model_names = [
    "raft", "pwcnet", "gma", "rpknet", "ccmr", "craft", "dicl", "dip", 
    "fastflownet", "maskflownet", "maskflownet_s", "flow1d", "flowformer", 
    "flowformer++", "gmflow", "hd3", "liteflownet", 
    "liteflownet2", "liteflownet3", "llaflow", 
    "ms_raft+", "rapidflow", "scopeflow", 
    "skflow", "starflow", "videoflow_bof"
]

kitti_model_names_top_9 = [
    "raft", "rpknet", "craft", "maskflownet", "flow1d", "flowformer++", 
    "liteflownet3_pseudoreg", "ms_raft+", 
    "scopeflow", "gma"
]

sintel_model_names_top_9 = [
    "raft", "rpknet", "craft", "maskflownet", "flow1d", "flowformer++", 
    "liteflownet3", "ms_raft+", 
    "scopeflow", "gma"
]

import pandas as pd
import numpy as np

# Define the casting dictionary
attributes_to_cast = {
    'attack': str,
    'targeted': 'boolean',  # Can use a boolean cast, True/False or None
    'iterations': float,
    'alpha': float,
    'loss': str,
    'epsilon': float,
    'target': str,
    'norm': str,
    'name': str,
    'severity': float
}

# Function to cast columns in merged_df
def cast_columns(merged_df, attributes_to_cast):
    for col, dtype in attributes_to_cast.items():
        if dtype == 'boolean':
            merged_df[col] = merged_df[col].astype('bool', errors='ignore')
        else:
            merged_df[col] = merged_df[col].astype(dtype, errors='ignore')
    return merged_df

# Cast the columns
merged_df = cast_columns(merged_df, attributes_to_cast)

# Function to match combinations
def combination_matches_row(input_df, combination_list, output_df, model, dataset):
    # Iterate over each combination in the combination list
    for combination in combination_list:
        combination_in_input_df = False

        # **Check if "optim" exists in combination and restrict it to top 9 models**
        if "optim" in combination.keys() and combination["optim"] == "initial_flow":
            if dataset == "kitti-2015":
                if model not in kitti_model_names_top_9:
                    continue  # Skip this combination if the model is not in top 9 for kitti-2015
            else:
                if model not in sintel_model_names_top_9:
                    continue  # Skip this combination if the model is not in top 9 for other datasets

        # **Check for common corruptions only for kitti-2015 dataset**
        #if combination['attack'] == 'common_corruptions' and dataset != 'kitti-2015':
            #continue  # Skip common corruptions if not kitti-2015 dataset

        # Iterate over each row in the dataframe
        for index, row in input_df.iterrows():
            row_in_combination = True  # Assume the row matches the combination initially
            row_dict = row.to_dict()

            # Loop through each key-value pair in the combination
            for key, value in combination.items():
                if key not in row_dict.keys() or row_dict[key] != value:
                    row_in_combination = False
                    break  # Exit the inner loop because we know this row doesn't match

            # If row_in_combination is still True after the inner loop, it means the row matches the combination
            if row_in_combination:
                combination_in_input_df = True
                break  # No need to continue checking rows, as we found a match

        # If no matching row was found, add the combination to output_df
        if not combination_in_input_df:
            combination["model"] = model
            combination["dataset"] = dataset
            output_df = pd.concat([output_df, pd.DataFrame([combination])], ignore_index=True)
            del combination["model"]
            del combination["dataset"]

    return output_df

# Create an empty dataframe to store missing combinations
missing_comb_df = pd.DataFrame()

# Iterate through each dataset
for dataset in datasets:
    # Select the relevant model names based on the dataset
    if "kitti" in dataset:
        model_names = kitti_model_names
    else:
        model_names = sintel_model_names
    
    # Iterate through each model in the dataset
    for model in model_names:
        # Filter the dataframe for the current dataset and model
        subset_df = merged_df[(merged_df['dataset'] == dataset) & (merged_df['model'] == model)]
        missing_comb_df = combination_matches_row(subset_df, combinations_list, missing_comb_df, model, dataset)

# Rearrange columns to have 'model' and 'dataset' first
columns_order = ['model', 'dataset'] + [col for col in missing_comb_df.columns if col not in ['model', 'dataset']]
missing_comb_df = missing_comb_df[columns_order]

# Sort by 'model', 'dataset', 'norm', and 'attack'
if 'norm' in missing_comb_df.columns:
    missing_comb_df = missing_comb_df.sort_values(by=['model', 'dataset','norm', 'attack'], ascending=True)
else:
    missing_comb_df = missing_comb_df.sort_values(by=['model', 'dataset', 'attack'], ascending=True)

# Save the missing combinations dataframe to a CSV file
missing_comb_df.to_csv('missing_combinations.csv', index=False)

print("Missing combinations saved to 'missing_combinations.csv'")




Missing combinations saved to 'missing_combinations.csv'


In [2]:
import numpy as np

combinations_list = [
    # Targeted: True, epsilon: 0.0314, target: 'negative' or 'zero' (for attacks: pgd, cospgd, bim, norm: 'inf')
    {'attack': 'pgd', 'targeted': True, 'iterations': 20.0, 'alpha': 0.01, 'loss': 'epe', 'epsilon': 0.0314, 'target': 'negative', 'norm': 'inf'},
    {'attack': 'pgd', 'targeted': True, 'iterations': 20.0, 'alpha': 0.01, 'loss': 'epe', 'epsilon': 0.0314, 'target': 'zero', 'norm': 'inf'},
    {'attack': 'cospgd', 'targeted': True, 'iterations': 20.0, 'alpha': 0.01, 'loss': 'epe', 'epsilon': 0.0314, 'target': 'negative', 'norm': 'inf'},
    {'attack': 'cospgd', 'targeted': True, 'iterations': 20.0, 'alpha': 0.01, 'loss': 'epe', 'epsilon': 0.0314, 'target': 'zero', 'norm': 'inf'},
    {'attack': 'bim', 'targeted': True, 'iterations': 20.0, 'alpha': 0.01, 'loss': 'epe', 'epsilon': 0.0314, 'target': 'negative', 'norm': 'inf'},
    {'attack': 'bim', 'targeted': True, 'iterations': 20.0, 'alpha': 0.01, 'loss': 'epe', 'epsilon': 0.0314, 'target': 'zero', 'norm': 'inf'},

    # Targeted: True, epsilon: 0.2510, target: 'negative' or 'zero' (for attacks: pgd, cospgd, bim, norm: 'two')
    {'attack': 'pgd', 'targeted': True, 'iterations': 20.0, 'alpha': 0.1, 'loss': 'epe', 'epsilon': 0.2510, 'target': 'negative', 'norm': 'two'},
    {'attack': 'pgd', 'targeted': True, 'iterations': 20.0, 'alpha': 0.1, 'loss': 'epe', 'epsilon': 0.2510, 'target': 'zero', 'norm': 'two'},
    {'attack': 'cospgd', 'targeted': True, 'iterations': 20.0, 'alpha': 0.1, 'loss': 'epe', 'epsilon': 0.2510, 'target': 'negative', 'norm': 'two'},
    {'attack': 'cospgd', 'targeted': True, 'iterations': 20.0, 'alpha': 0.1, 'loss': 'epe', 'epsilon': 0.2510, 'target': 'zero', 'norm': 'two'},
    {'attack': 'bim', 'targeted': True, 'iterations': 20.0, 'alpha': 0.1, 'loss': 'epe', 'epsilon': 0.2510, 'target': 'negative', 'norm': 'two'},
    {'attack': 'bim', 'targeted': True, 'iterations': 20.0, 'alpha': 0.1, 'loss': 'epe', 'epsilon': 0.2510, 'target': 'zero', 'norm': 'two'},

    # Targeted: True, epsilon: 0.0314, target: 'negative' or 'zero' (for attack: fgsm, norm: 'inf')
    {'attack': 'fgsm', 'targeted': True, 'loss': 'epe', 'epsilon': 0.0314, 'target': 'negative', 'norm': 'inf'},
    {'attack': 'fgsm', 'targeted': True, 'loss': 'epe', 'epsilon': 0.0314, 'target': 'zero', 'norm': 'inf'},
    {'attack': 'fgsm', 'targeted': True, 'loss': 'epe', 'epsilon': 0.0, 'target': 'negative', 'norm': 'inf'},
    {'attack': 'fgsm', 'targeted': True, 'loss': 'epe', 'epsilon': 0.0, 'target': 'zero', 'norm': 'inf'},

    # Targeted: True, epsilon: 0.2510, target: 'negative' or 'zero' (for attack: fgsm, norm: 'two')
    {'attack': 'fgsm', 'targeted': True, 'loss': 'epe', 'epsilon': 0.2510, 'target': 'negative', 'norm': 'two'},
    {'attack': 'fgsm', 'targeted': True, 'loss': 'epe', 'epsilon': 0.2510, 'target': 'zero', 'norm': 'two'},
    {'attack': 'fgsm', 'targeted': True, 'loss': 'epe', 'epsilon': 0.0, 'target': 'negative', 'norm': 'two'},
    {'attack': 'fgsm', 'targeted': True, 'loss': 'epe', 'epsilon': 0.0, 'target': 'zero', 'norm': 'two'},

    # Targeted: False, epsilon: 0.0314 or 0.0157, target: None (for attacks: pgd, cospgd, bim, norm: 'inf')
    {'attack': 'pgd', 'targeted': False, 'iterations': 20.0, 'alpha': 0.01, 'loss': 'epe', 'epsilon': 0.0314, 'norm': 'inf'},
    {'attack': 'pgd', 'targeted': False, 'iterations': 20.0, 'alpha': 0.01, 'loss': 'epe', 'epsilon': 0.0157, 'norm': 'inf'},
    {'attack': 'cospgd', 'targeted': False, 'iterations': 20.0, 'alpha': 0.01, 'loss': 'epe', 'epsilon': 0.0314, 'norm': 'inf'},
    {'attack': 'cospgd', 'targeted': False, 'iterations': 20.0, 'alpha': 0.01, 'loss': 'epe', 'epsilon': 0.0157, 'norm': 'inf'},
    {'attack': 'bim', 'targeted': False, 'iterations': 20.0, 'alpha': 0.01, 'loss': 'epe', 'epsilon': 0.0314, 'norm': 'inf'},
    {'attack': 'bim', 'targeted': False, 'iterations': 20.0, 'alpha': 0.01, 'loss': 'epe', 'epsilon': 0.0157, 'norm': 'inf'},

    # Targeted: False, epsilon: 0.2510, target: None (for attacks: pgd, cospgd, bim, norm: 'two')
    {'attack': 'pgd', 'targeted': False, 'iterations': 20.0, 'alpha': 0.1, 'loss': 'epe', 'epsilon': 0.2510, 'norm': 'two'},
    {'attack': 'cospgd', 'targeted': False, 'iterations': 20.0, 'alpha': 0.1, 'loss': 'epe', 'epsilon': 0.2510, 'norm': 'two'},
    {'attack': 'bim', 'targeted': False, 'iterations': 20.0, 'alpha': 0.1, 'loss': 'epe', 'epsilon': 0.2510, 'norm': 'two'},

    #Top 9 models configuration
    {'targeted': False, 'attack': 'cospgd', 'epsilon': 0.0314, 'alpha': 0.01, 'iterations': 20.0, 'norm': 'inf', 'loss': 'epe', 'optim': "initial_flow"},
    {'targeted': False, 'attack': 'pgd', 'epsilon': 0.0314, 'alpha': 0.01, 'iterations': 20.0, 'norm': 'inf', 'loss': 'epe', 'optim': "initial_flow"},

    # Targeted: False, epsilon: 0.0314 or 0.0157, target: None (for attack: fgsm, norm: 'inf')
    {'attack': 'fgsm', 'targeted': False, 'loss': 'epe', 'epsilon': 0.0314, 'norm': 'inf'},
    {'attack': 'fgsm', 'targeted': False, 'loss': 'epe', 'epsilon': 0.0157, 'norm': 'inf'},

    # Targeted: False, epsilon: 0.2510, target: None (for attack: fgsm, norm: 'two')
    {'attack': 'fgsm', 'targeted': False, 'loss': 'epe', 'epsilon': 0.2510, 'norm': 'two'},

    # PCFA
    {'attack': 'pcfa', 'iterations': 20.0, 'epsilon': 0.05, 'target': 'negative'},
    {'attack': 'pcfa', 'iterations': 20.0, 'epsilon': 0.05, 'target': 'zero'},

    {'attack': 'common_corruptions', 'name': 'gaussian_noise', 'severity': 3.0},
    {'attack': 'common_corruptions', 'name': 'shot_noise', 'severity': 3.0},
    {'attack': 'common_corruptions', 'name': 'impulse_noise', 'severity': 3.0},
    {'attack': 'common_corruptions', 'name': 'defocus_blur', 'severity': 3.0},
    {'attack': 'common_corruptions', 'name': 'glass_blur', 'severity': 3.0},
    {'attack': 'common_corruptions', 'name': 'motion_blur', 'severity': 3.0},
    {'attack': 'common_corruptions', 'name': 'zoom_blur', 'severity': 3.0},
    {'attack': 'common_corruptions', 'name': 'snow', 'severity': 3.0},
    {'attack': 'common_corruptions', 'name': 'frost', 'severity': 3.0},
    {'attack': 'common_corruptions', 'name': 'fog', 'severity': 3.0},
    {'attack': 'common_corruptions', 'name': 'brightness', 'severity': 3.0},
    {'attack': 'common_corruptions', 'name': 'contrast', 'severity': 3.0},
    {'attack': 'common_corruptions', 'name': 'elastic_transform', 'severity': 3.0},
    {'attack': 'common_corruptions', 'name': 'pixelate', 'severity': 3.0},
    {'attack': 'common_corruptions', 'name': 'jpeg_compression', 'severity': 3.0},

    {'attack': '3dcc', '3dcc_corruption': 'far_focus', '3dcc_intensity': 3.0},
    {'attack': '3dcc', '3dcc_corruption': 'near_focus', '3dcc_intensity': 3.0},
    {'attack': '3dcc', '3dcc_corruption': 'fog_3d', '3dcc_intensity': 3.0},
    {'attack': '3dcc', '3dcc_corruption': 'color_quant', '3dcc_intensity': 3.0},
    {'attack': '3dcc', '3dcc_corruption': 'iso_noise', '3dcc_intensity': 3.0},
    {'attack': '3dcc', '3dcc_corruption': 'low_light', '3dcc_intensity': 3.0},
    {'attack': '3dcc', '3dcc_corruption': 'xy_motion_blur', '3dcc_intensity': 3.0},
    {'attack': '3dcc', '3dcc_corruption': 'z_motion_blur', '3dcc_intensity': 3.0},

    {'attack': 'weather', 'targeted': False, 'flakesize': 71.0, 'depth': 9.0, 'transparency': 0.75, 'do': 0.0, 'weather_type': 'snow'},
    {'attack': 'weather', 'targeted': True, 'target': 'negative', 'flakesize': 71.0, 'depth': 9.0, 'transparency': 0.75, 'do': 0.0, 'weather_type': 'snow'},
    {'attack': 'weather', 'targeted': True, 'target': 'zero', 'flakesize': 71.0, 'depth': 9.0, 'transparency': 0.75, 'do': 0.0, 'weather_type': 'snow'},

    {'attack': 'weather', 'targeted': False, 'num': 1000.0, 'flakesize': 51.0, 'depth': 9.0, 'transparency': 0.75, 'do': 1.0,  'weather_type': 'rain'},
    {'attack': 'weather', 'targeted': True, 'target': 'negative',  'num': 1000.0, 'flakesize': 51.0, 'depth': 9.0, 'transparency': 0.75, 'do': 1.0, 'weather_type': 'rain'},
    {'attack': 'weather', 'targeted': True,'target': 'zero',  'num': 1000.0, 'flakesize': 51.0, 'depth': 9.0, 'transparency': 0.75, 'do': 1.0, 'weather_type': 'rain'},

    {'attack': 'weather', 'targeted': False, 'num': 60.0, 'flakesize': 451.0, 'depth': 0.8, 'constant': 0.3, 'transparency': 0.25, 'do': 0.0,  'weather_type': 'fog'},
    {'attack': 'weather', 'targeted': True, 'target': 'negative',  'num': 60.0, 'flakesize': 451.0, 'depth': 0.8, 'constant': 0.3, 'transparency': 0.25, 'do': 0.0, 'weather_type': 'fog'},
    {'attack': 'weather', 'targeted': True, 'target': 'zero',  'num': 60.0, 'flakesize': 451.0, 'depth': 0.8, 'constant': 0.3, 'transparency': 0.25, 'do': 0.0, 'weather_type': 'fog'},
    
    {'attack': 'weather', 'targeted': False, 'num': 3000.0, 'flakesize': 41.0, 'depth': 9.0, 'transparency': 1.5, 'do': 1.0,  'weather_type': 'sparks'},
    {'attack': 'weather', 'targeted': True, 'target': 'negative',  'num': 3000.0, 'flakesize': 41.0, 'depth': 9.0, 'transparency': 1.5, 'do': 1.0, 'weather_type': 'sparks'},
    {'attack': 'weather', 'targeted': True, 'target': 'zero',  'num': 3000.0, 'flakesize': 41.0, 'depth': 9.0, 'transparency': 1.5, 'do': 1.0, 'weather_type': 'sparks'},

    {'attack': 'none'},

]

# Set the dataset and norm
datasets = ["sintel-final", "sintel-clean", "kitti-2015"]

# Define the datasets and corresponding model names
kitti_model_names = [
    "raft", "gma", "rpknet", "ccmr", "craft", "csflow", "dicl", "dip", 
    "fastflownet", "maskflownet", "flow1d", "flowformer", "flowformer++", 
    "gmflow", "gmflownet", "hd3", "irr_pwc", "liteflownet", 
    "liteflownet3_pseudoreg", "llaflow", "ms_raft+", 
    "rapidflow", "scopeflow", "skflow", 
    "starflow", "videoflow_bof", "splatflow"
]

sintel_model_names = [
    "raft", "pwcnet", "gma", "rpknet", "ccmr", "craft", "dicl", "dip", 
    "fastflownet", "maskflownet", "maskflownet_s", "flow1d", "flowformer", 
    "flowformer++", "gmflow", "hd3", "liteflownet", 
    "liteflownet2", "liteflownet3", "llaflow", 
    "ms_raft+", "rapidflow", "scopeflow", 
    "skflow", "starflow", "videoflow_bof", "neuflow"
]

kitti_model_names_top_9 = [
    "raft", "rpknet", "craft", "maskflownet", "flow1d", "flowformer++", 
    "liteflownet3_pseudoreg", "ms_raft+", 
    "scopeflow", "gma"
]

sintel_model_names_top_9 = [
    "raft", "rpknet", "craft", "maskflownet", "flow1d", "flowformer++", 
    "liteflownet3", "ms_raft+", 
    "scopeflow", "gma"
]

import pandas as pd
import numpy as np

# Define the casting dictionary
attributes_to_cast = {
    'attack': str,
    'targeted': 'boolean',  # Can use a boolean cast, True/False or None
    'iterations': float,
    'alpha': float,
    'loss': str,
    'epsilon': float,
    'target': str,
    'norm': str,
    'name': str,
    'severity': float
}

# Function to cast columns in merged_df
def cast_columns(merged_df, attributes_to_cast):
    for col, dtype in attributes_to_cast.items():
        if dtype == 'boolean':
            merged_df[col] = merged_df[col].astype('bool', errors='ignore')
        else:
            merged_df[col] = merged_df[col].astype(dtype, errors='ignore')
    return merged_df

# Cast the columns
merged_df = cast_columns(merged_df, attributes_to_cast)

def filter_and_append_rows_by_combination(input_df, combination_list, output_df):
    # Iterate over each combination in the combination list
    for combination in combination_list:
        # Initialize the mask to True for all rows
        combination_mask = pd.Series([True] * len(input_df), index=input_df.index)

        # Loop through each key-value pair in the combination
        for key, value in combination.items():
            if key in input_df.columns:
                # Update the mask for the current combination
                combination_mask &= (input_df[key] == value)

        # Filter the input_df to keep only the matching rows for the current combination
        filtered_df = input_df[combination_mask].reset_index(drop=True)

        # Concatenate filtered_df with output_df
        output_df = pd.concat([output_df, filtered_df], ignore_index=True)

    return output_df



# Create an empty dataframe to store missing combinations
relevant_experiments = pd.DataFrame()

# Iterate through each dataset
for dataset in datasets:
    # Select the relevant model names based on the dataset
    if "kitti" in dataset:
        model_names = kitti_model_names
    else:
        model_names = sintel_model_names
    
    # Iterate through each model in the dataset
    for model in model_names:
        
        # Filter the dataframe for the current dataset and model
        subset_df = merged_df[(merged_df['dataset'] == dataset) & (merged_df['model'] == model)]

        relevant_experiments = filter_and_append_rows_by_combination(subset_df, combinations_list, relevant_experiments)

# Rearrange columns to have 'model' and 'dataset' first
columns_order = ['model', 'dataset'] + [col for col in relevant_experiments.columns if col not in ['model', 'dataset']]
relevant_experiments = relevant_experiments[columns_order]

# Sort by 'model', 'dataset', 'norm', and 'attack'
relevant_experiments = relevant_experiments.sort_values(by=['model', 'dataset', 'norm', 'attack'], ascending=True)

with open('model_params.json', 'r') as file:
    data = json.load(file)

# Convert the JSON data into a DataFrame
df_params = pd.DataFrame(data['experiment'])

final_df = pd.merge(left=relevant_experiments, right=df_params, on="model")

# Save the missing combinations dataframe to a CSV file
final_df.to_csv('existing_relevant_combinations.csv', index=False)

print("Relevant combinations saved to 'existing_relevant_combinations.csv'")

Relevant combinations saved to 'existing_relevant_combinations.csv'


In [18]:
import pandas as pd

df = pd.read_csv("existing_relevant_combinations.csv", delimiter=",", keep_default_na=False)

relevant_columns = ["model", "dataset", "attack", "norm", "epsilon", 
          "target", "epe", "px1", "px3", "px5", "cosim", "outlier", "l2_delta12", 
          "l0_delta12", "l_inf_delta12", "epe_orig_preds", "cosim_target", "cosim_orig_preds", 
          "epe_ground_truth_to_negative", "epe_ground_truth_to_zero", "name", "epe_ground_truth", 
          "3dcc_corruption", "model_parameters", "point_matching_method"]

df = df[relevant_columns]
df["attack"] = df["attack"].replace("none", "clean")

# for col in final_df.columns:
#     print(col)

final_df

,model,dataset,checkpoint,attack,norm,epsilon,iterations,alpha,targeted,target,...,num,weather_type,start_time,end_time,duration,epe_initial_to_zero,epe_initial_to_negative,model_parameters,point_matching_method,time
0,ccmr,kitti-2015,kitti,bim,inf,0.0314,20.0,0.01,True,negative,...,NaN,NaN,2024-09-11 21:46:28,2024-09-12 00:16:01,02:29:33,31.192852,62.385703,10780884.0,attention,01/2024
1,ccmr,kitti-2015,kitti,bim,inf,0.0314,20.0,0.01,True,zero,...,NaN,NaN,2024-09-11 21:48:27,2024-09-12 00:16:24,02:27:57,31.192852,62.385703,10780884.0,attention,01/2024
2,ccmr,kitti-2015,kitti,bim,inf,0.0314,20.0,0.01,False,nan,...,NaN,NaN,2024-07-09 12:55:28,2024-07-09 14:38:33,01:43:04,31.192852,62.385703,10780884.0,attention,01/2024
3,ccmr,kitti-2015,kitti,bim,inf,0.0314,20.0,0.01,False,nan,...,NaN,NaN,2024-09-11 23:19:34,2024-09-12 01:51:06,02:31:31,31.192852,62.385703,10780884.0,attention,01/2024
4,ccmr,kitti-2015,kitti,bim,inf,0.0314,20.0,0.01,False,nan,...,NaN,NaN,2024-07-09 12:55:28,2024-07-09 14:38:33,01:43:04,31.192852,62.385703,10780884.0,attention,01/2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5093,videoflow_bof,sintel-final,sintel,fgsm,two,0.0000,NaN,NaN,True,zero,...,NaN,NaN,2024-09-19 14:15:47,2024-09-19 14:27:20,00:00:00,NaN,NaN,12659389.0,correlation,03/2023
5094,videoflow_bof,sintel-final,sintel,fgsm,two,0.2510,NaN,NaN,False,nan,...,NaN,NaN,2024-09-14 16:19:26,2024-09-14 17:09:38,00:50:11,NaN,NaN,12659389.0,correlation,03/2023
5095,videoflow_bof,sintel-final,sintel,pgd,two,0.2510,20.0,0.10,True,negative,...,NaN,NaN,2024-09-12 21:18:15,2024-09-13 03:55:23,06:37:08,NaN,NaN,12659389.0,correlation,03/2023
5096,videoflow_bof,sintel-final,sintel,pgd,two,0.2510,20.0,0.10,True,zero,...,NaN,NaN,2024-09-12 21:18:15,2024-09-13 03:55:19,06:37:04,NaN,NaN,12659389.0,correlation,03/2023


In [27]:
# Example function to reshape the DataFrame for a single attack
def reshape_for_attack(df, attack_name):
    # Filter rows for the specific attack
    attack_df = df[df['attack'] == attack_name].copy()
    
    # Drop unnecessary columns if needed
    attack_df = attack_df.drop(columns=['attack'], errors='ignore')
    
    # Create a pivot table with a multi-level column index
    pivot = attack_df.pivot_table(
        index=['model', 'dataset'], 
        columns=['norm', 'epsilon', 'target', 'name', '3dcc_corruption'],
        aggfunc='first'
    )
    
    # Flatten the multi-index columns and create unique names
    pivot.columns = [
        f"{attack_name}_{'_'.join(map(str, col))}" for col in pivot.columns
    ]
    
    # Reset the index for merging later
    pivot.reset_index(inplace=True)
    return pivot

# Combine results for all attacks
def transform_dataframe(df, attacks):
    result = pd.DataFrame()
    for attack in attacks:
        attack_pivot = reshape_for_attack(df, attack)
        if result.empty:
            result = attack_pivot
        else:
            result = pd.merge(result, attack_pivot, on=['model', 'dataset'], how='outer')
    return result


attacks = df['attack'].unique()  # Get unique attack names
transformed_df = transform_dataframe(df, attacks)

transformed_df.to_csv('transformed_data.csv', index=False)

transformed_df


/scratch/ipykernel_4039002/860694991.py:40: DtypeWarning: Columns (294) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("existing_relevant_combinations.csv", delimiter=",", keep_default_na=False)


,model,dataset,bim_checkpoint_inf_0.0157_nan_nan_,bim_checkpoint_inf_0.0314_nan_nan_,bim_checkpoint_inf_0.0314_negative_nan_,bim_checkpoint_inf_0.0314_zero_nan_,bim_checkpoint_two_0.251_nan_nan_,bim_checkpoint_two_0.251_negative_nan_,bim_checkpoint_two_0.251_zero_nan_,bim_cosim_inf_0.0157_nan_nan_,...,weather_outlier_nan__negative_nan_,weather_outlier_nan__zero_nan_,weather_point_matching_method_nan__negative_nan_,weather_point_matching_method_nan__zero_nan_,weather_px1_nan__negative_nan_,weather_px1_nan__zero_nan_,weather_px3_nan__negative_nan_,weather_px3_nan__zero_nan_,weather_px5_nan__negative_nan_,weather_px5_nan__zero_nan_
0,ccmr,kitti-2015,kitti,kitti,kitti,kitti,kitti,kitti,kitti,0.1804731972562149,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ccmr,sintel-clean,sintel,sintel,sintel,sintel,sintel,sintel,sintel,0.9011316607553265,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ccmr,sintel-final,sintel,sintel,sintel,sintel,sintel,sintel,sintel,0.8720889007395328,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,craft,kitti-2015,kitti,kitti,kitti,kitti,kitti,kitti,kitti,0.17133171987719834,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,craft,sintel-clean,sintel,sintel,sintel,sintel,sintel,sintel,sintel,0.4995292919335885,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,starflow,sintel-clean,sintel,sintel,sintel,sintel,sintel,sintel,sintel,0.8399587799575785,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
76,starflow,sintel-final,sintel,sintel,sintel,sintel,sintel,sintel,sintel,0.7988400316624866,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
77,videoflow_bof,kitti-2015,kitti,kitti,NaN,NaN,kitti,NaN,NaN,0.16907726036035456,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
78,videoflow_bof,sintel-clean,sintel,sintel,sintel,sintel,sintel,sintel,sintel,0.4640022494974061,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
for col in transformed_df.columns:
    print(col)

model
dataset
bim_checkpoint_inf_0.0157_nan_nan_
bim_checkpoint_inf_0.0314_nan_nan_
bim_checkpoint_inf_0.0314_negative_nan_
bim_checkpoint_inf_0.0314_zero_nan_
bim_checkpoint_two_0.251_nan_nan_
bim_checkpoint_two_0.251_negative_nan_
bim_checkpoint_two_0.251_zero_nan_
bim_cosim_inf_0.0157_nan_nan_
bim_cosim_inf_0.0314_nan_nan_
bim_cosim_inf_0.0314_negative_nan_
bim_cosim_inf_0.0314_zero_nan_
bim_cosim_two_0.251_nan_nan_
bim_cosim_two_0.251_negative_nan_
bim_cosim_two_0.251_zero_nan_
bim_cosim_orig_preds_inf_0.0157_nan_nan_
bim_cosim_orig_preds_inf_0.0314_nan_nan_
bim_cosim_orig_preds_inf_0.0314_negative_nan_
bim_cosim_orig_preds_inf_0.0314_zero_nan_
bim_cosim_orig_preds_two_0.251_nan_nan_
bim_cosim_orig_preds_two_0.251_negative_nan_
bim_cosim_orig_preds_two_0.251_zero_nan_
bim_cosim_target_inf_0.0157_nan_nan_
bim_cosim_target_inf_0.0314_nan_nan_
bim_cosim_target_inf_0.0314_negative_nan_
bim_cosim_target_inf_0.0314_zero_nan_
bim_cosim_target_two_0.251_nan_nan_
bim_cosim_target_two_0.251_